In [2]:
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys 
sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import importlib
import scipy.io 
from scipy.signal import find_peaks
import matlab.engine
import tqdm 
import warnings
import virtual_foveation
import hdf5storage
import dill 
import cv2

In [ ]:
importlib.reload(virtual_foveation)
virtual_foveation.find_sensor_start_end_times("/Users/zacharykelly/Desktop/spatialFrequency")

TypeError: object of type 'generator' has no len()

In [ ]:
# Generate a playable video
path_to_recording_chunks: str = "/Volumes/T7 Shield/sam_gazecal_106"
output_dir: str = "./"
Pi_util.generate_playable_videos(path_to_recording_chunks, output_dir, apply_digital_gain=True, fill_missing_frames=True, debayer_images=True, pupil_image_rotation_correction=True)

In [ ]:
# Load in the frames of the recording 
world_frames: np.ndarray = Pi_util.destruct_video("./sam_gazecal_106/W.avi", is_grayscale=True)
print(world_frames.shape)

In [ ]:
plt.imshow(world_frames[21000], cmap='gray')

In [ ]:
frame_idx: np.ndarray = np.array([i for i in range(15000, 16000, 100)])

In [ ]:
for frame in world_frames[frame_idx]:
    plt.imshow(frame, cmap='gray')
    plt.show()


In [ ]:
scipy.io.savemat("sam106GazeCal_testframes.mat", {"frames": world_frames[frame_idx]}) # 15000

In [ ]:
# Allocate an array of transformed frames 
transformed_world_frames: np.ndarray = np.zeros_like(world_frames)

In [ ]:
# Load in the pupil gaze angles 
gaze_angles: np.ndarray = np.nan_to_num(scipy.io.loadmat("./gaze_angles")["gaze_angles"], 0)
print(gaze_angles.shape)


In [ ]:
scipy.io.savemat("sam106GazeCal_gaze_angles.mat", {"gaze_angles": gaze_angles[frame_idx - 65]})

In [ ]:
# Load in the first chunk of both cameras. This will tell us if one or the other started first 
world_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/world_time_2025DASH10DASH06_09COLON26COLON14DOT693525_chunk_0_metadata.npy")
pupil_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/pupil_time_2025DASH10DASH06_09COLON26COLON15DOT252214_chunk_0_metadata.npy")

In [ ]:
# Find the missing number of frames between the two measurements in time 
FPS: float = 120
missing_frames: int = (abs(len(world_frames) - len(gaze_angles)))
missing_time: float = missing_frames / FPS 
print(missing_frames)
print(missing_time)

# Print the start time of both cameras so we can see whcih is first 
print(world_start_chunk_metadata[0, 0] / (10 ** 9))
print(pupil_start_chunk_metadata[0, 0])

# Find the difference explained by this 
start_time_delay_time: float = abs(pupil_start_chunk_metadata[0, 0] - world_start_chunk_metadata[0, 0] / (10 ** 9))
start_time_delay_frames: int = int(np.ceil(start_time_delay_time * FPS))
print(start_time_delay_time)
print(start_time_delay_frames)

# Account for the fact that we measured the pupil camera time wise is 0.005 phase advanced 
pupil_phase_offset_seconds: float = 0.005
start_time_delay_time -= pupil_phase_offset_seconds
start_time_delay_frames -= int(np.ceil(pupil_phase_offset_seconds * FPS))

In [ ]:
# Start a new MATLAB session
eng: object = matlab.engine.start_matlab()
eng.tbUseProject("lightLoggerAnalysis", nargout=0)


In [ ]:
# Initialize the objects we will use 
intrinsics_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/intrinsics_calibration.mat"
transformation_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/perspective_transform"

intrinsics: dict = scipy.io.loadmat(intrinsics_path)["camera_intrinsics_calibration"]
transformation: dict = scipy.io.loadmat(transformation_path)


In [ ]:
print(transformation["perspective_transform"])

In [ ]:
# Iterate over the world frames 
frames_for_video: int = len(world_frames)
frames_transformed = []
for world_frame_num in tqdm.trange(start_time_delay_frames, frames_for_video):    
    print(f"Processing frame: {world_frame_num+1}/{frames_for_video}", flush=True)

    # Retrieve the pupil frame that corresponds to this world camera frame 
    pupil_frame_num: int = world_frame_num - start_time_delay_frames

    # If we have gone out of bounds for pupil, simply quit 
    if(pupil_frame_num >= len(gaze_angles)):
        warnings.warn(f"{pupil_frame_num} Out of bounds for pupil frame, breaking")
        break 

    # Otherwise, retrieve the gaze angles for this pupil frame 
    # and world frame 
    world_frame: np.ndarray = world_frames[world_frame_num]
    pupil_gaze_angles: np.ndarray = gaze_angles[pupil_frame_num, :2]
    pupil_gaze_angles[0] = -pupil_gaze_angles[0]
    pupil_gaze_angles[1] -= 5.4

    # If there is nan gaze angles, just skip this frame 
    # and leave it all 0s
    if(np.any(np.isnan(pupil_gaze_angles))):
        continue

    # Then feed this as input into MATLAB to generate the correced frame 
    transformed_world_frame: np.ndarray = np.array(eng.coordinateTransformFinal(matlab.double(np.ascontiguousarray(world_frame).astype(np.float64)),
                                                                                matlab.double(np.ascontiguousarray(pupil_gaze_angles.astype(np.float64))),
                                                                                intrinsics_path, 
                                                                                transformation_path,
                                                                                nargout=1
                                                                               ))
    
    frames_transformed.append(transformed_world_frame)


In [ ]:
# Close the MATLAB engine 
eng.close() 

In [ ]:

Pi_util.frames_to_video(padded_frames, output_path="virtually_foveated_video.avi", fps=120)

In [ ]:
with open("inhomogenous_frames.pkl", "wb") as f:
    dill.dump(frames_transformed, f)

In [ ]:
frames_test = None 
with open("inhomogenous_frames.pkl", "rb") as f:
    frames_test = dill.load(f)

In [ ]:
import collections 

frame_size_frequency_counter: collections.Counter = collections.Counter([frame.shape for frame in frames_transformed])
max_size, count = frame_size_frequency_counter.most_common(1)[0]

In [ ]:
# Next, we will iterate through the frames and pad the ones that are not max size 
padded_frames: np.ndarray = np.zeros((len(frames_transformed), *max_size[:2]), dtype=np.uint8)

In [ ]:
def pad_frame(frame, target_shape, constant_value=0):
    """
    Pad a frame (H×W×C) to target_shape=(target_H, target_W[, C])
    using constant padding (default black).
    """
    h, w = frame.shape[:2]
    target_h, target_w = target_shape[:2]

    # Compute padding for each side
    pad_top = (target_h - h) // 2
    pad_bottom = target_h - h - pad_top
    pad_left = (target_w - w) // 2
    pad_right = target_w - w - pad_left

    # Apply padding
    if frame.ndim == 3:
        padded = np.pad(frame,
                        ((pad_top, pad_bottom),
                         (pad_left, pad_right),
                         (0, 0)),
                        mode='constant',
                        constant_values=constant_value)
    else:
        padded = np.pad(frame,
                        ((pad_top, pad_bottom),
                         (pad_left, pad_right)),
                        mode='constant',
                        constant_values=constant_value)
    return padded

In [ ]:
for frame_idx in tqdm.tqdm(range(len(frames_transformed))):
    try:
        padded_frames[frame_idx] = cv2.cvtColor( frames_transformed[frame_idx], cv2.COLOR_RGB2GRAY )
    except:
        continue


In [ ]:
# Find solely the inhomogenously sized frames 
inhomogenous_frames: list = [frame for frame in frames_test if frame.shape != max_size]

In [ ]:
len(inhomogenous_frames)/len(frames_test)

In [ ]:
for frame in inhomogenous_frames:
    plt.imshow(frame, cmap='gray')
    plt.title(f"Size: {frame.shape}")
    plt.show()